# Екзаменаційна робота

Навка Гліб Олександрович, ІПС-41

## Завдання 1

Аналіз набору даних за допомогою Pandas, NumPy, візуалізації даних за допомогою Matplotlib у блокноті Jupyter.

In [102]:
import pandas as pd

In [103]:
df = pd.read_csv("Expanded_data_with_more_features.csv")
df.head()

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


#### Проаналізуйте розподіл за статтю серед учнів

##### 1. Скільки студентів і студенток у наборі даних?

In [104]:
df["Gender"].value_counts()

Gender
female    15424
male      15217
Name: count, dtype: int64

##### 2. Який відсоток студентів чоловічої статі в наборі даних?

In [105]:
df["Gender"].value_counts(normalize=True)["male"] * 100

49.6622172905584

##### 3. Який відсоток студенток у наборі даних?

In [106]:
df["Gender"].value_counts(normalize=True)["female"] * 100

50.337782709441605

#### Дослідіть зв’язок між етнічною групою та результатами тестів

##### 4. Яка етнічна група має найвищий середній бал з математики?

In [107]:
ethnic_group_mean_math = df.groupby("EthnicGroup")["MathScore"].mean()
ethnic_group_mean_math.idxmax()

'group E'

##### 5. Яка етнічна група має найвищий середній бал читання?

In [108]:
ethnic_group_mean_reading = df.groupby("EthnicGroup")["ReadingScore"].mean()
ethnic_group_mean_reading.idxmax()

'group E'

##### 6. Яка етнічна група має найвищий середній бал з письма?

In [109]:
ethnic_group_mean_writing = df.groupby("EthnicGroup")["WritingScore"].mean()
ethnic_group_mean_writing.idxmax()

'group E'

#### Дослідіть вплив курсу підготовки до тесту на результати тесту

##### 7. Який середній бал з математики для студентів, які закінчили курс підготовки до тесту?

In [110]:
df_prep = df[df["TestPrep"] == "completed"]
df_prep["MathScore"].mean()

69.5466599698644

##### 8. Який середній бал з читання для студентів, які пройшли курс підготовки до тесту?

In [111]:
df_prep = df[df["TestPrep"] == "completed"]
df_prep["ReadingScore"].mean()

73.73299849321948

##### 9. Який середній бал для студентів, які закінчили курс підготовки до тесту?

In [112]:
df_test_prep_completed = df[df["TestPrep"] == "completed"]
df_test_prep_completed[["MathScore", "ReadingScore", "WritingScore"]].unstack().mean()

72.66097438473129

#### Проаналізуйте розподіл батьків(ів) за сімейним станом серед учнів

##### 10. Скільки студентів мають одружених батьків?

In [113]:
len(df[df["ParentMaritalStatus"] == "married"]["ParentMaritalStatus"])

16844

##### 11. Скільки студентів мають батьків-одинаків?

In [114]:
len(df[df["ParentMaritalStatus"] == "single"]["ParentMaritalStatus"])

7097

#### Дослідіть зв’язок між заняттями спортом і результатами тестів

##### 12. Чи студенти, які регулярно займаються спортом, мають вищі середні результати з математики, ніж ті, хто ніколи не займається спортом?

In [115]:
sport_math_students_mean = df.groupby("PracticeSport")["MathScore"].mean()

print(f"sport student average math score: {sport_math_students_mean['regularly']}")
print(f"non sport student average math scrore: {sport_math_students_mean['never']}")

sport_math_students_mean["regularly"] > sport_math_students_mean["never"]

sport student average math score: 67.83915500787548
non sport student average math scrore: 64.17107892107892


True

##### 13. Чи студенти, які регулярно займаються спортом, мають вищі середні результати читання, ніж ті, хто ніколи не займається спортом?

In [116]:
sport_reading_students_mean = df.groupby("PracticeSport")["ReadingScore"].mean()

print(f"sport student average math score: {sport_reading_students_mean['regularly']}")
print(f"non sport student average math scrore: {sport_reading_students_mean['never']}")

sport_reading_students_mean["regularly"] > sport_reading_students_mean["never"]

sport student average math score: 69.94301862318169
non sport student average math scrore: 68.33766233766234


True

#### Проаналізуйте розподіл транспортних засобів до школи серед учнів

##### 14. Скільки учнів користується шкільним автобусом, щоб дістатися до школи?

In [117]:
(df["TransportMeans"] == "school_bus").value_counts()[True]

16145

##### 15. Скільки учнів користується приватним транспортом, щоб дістатися до школи?

In [118]:
(df["TransportMeans"] == "private").value_counts()[True]

11362

___

## Завдання 2

Передбачте, чи завершив студент курс підготовки до іспиту, на основі його демографічних та академічних даних

Вхідні дані:
Gender, EthnicGroup, ParentEduc, LunchType, ParentMaritalStatus, PracticeSport, IsFirstChild, NrSiblings, TransportMeans, WklyStudyHours, MathScore, ReadingScore, WritingScore

Результат: TestPrep

In [119]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Підготовка даних

In [128]:
df = df.dropna()

x = df[[
    "Gender", 
    "EthnicGroup", 
    "ParentEduc", 
    "LunchType", 
    "ParentMaritalStatus", 
    "PracticeSport", 
    "IsFirstChild",
    "NrSiblings",
    "TransportMeans",
    "WklyStudyHours",
    "MathScore",
    "ReadingScore",
    "WritingScore"
]]

y = df["TestPrep"]

x_encoded = pd.get_dummies(x)
x_encoded.head()

,NrSiblings,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,EthnicGroup_group A,EthnicGroup_group B,EthnicGroup_group C,EthnicGroup_group D,...,PracticeSport_never,PracticeSport_regularly,PracticeSport_sometimes,IsFirstChild_no,IsFirstChild_yes,TransportMeans_private,TransportMeans_school_bus,WklyStudyHours_5 - 10,WklyStudyHours_< 5,WklyStudyHours_> 10
2,4.0,87,93,91,True,False,False,True,False,False,...,False,False,True,False,True,False,True,False,True,False
4,0.0,76,78,75,False,True,False,False,True,False,...,False,False,True,False,True,False,True,True,False,False
5,1.0,73,84,79,True,False,False,True,False,False,...,False,True,False,False,True,False,True,True,False,False
6,1.0,85,93,89,True,False,False,True,False,False,...,True,False,False,True,False,True,False,True,False,False
7,1.0,41,43,39,False,True,False,True,False,False,...,False,False,True,False,True,True,False,False,False,True


Розділення даних на тренувальні та тестові набори

In [121]:
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)

Ініціалізація та навчання моделі

In [126]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

Перевірка точності моделі

In [127]:
y_pred = rf_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
f"accuracy: {accuracy}"

'accuracy: 0.7113535983372304'